In [42]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graph

In [43]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [44]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

In [45]:
# Recreating vine-table
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R3EQSTM9PWRAL|          3|            0|          0|   N|                Y|
| RBWPRK17XKIXD|          5|            0|          0|   N|                Y|
| RRSLOAF273XFC|          5|            1|          2|   N|                Y|
|R3S8W9Q6SWIT8O|          4|            0|          0|   N|                Y|
|R3QQ6NSLRVBFJC|          4|            0|          0|   N|                Y|
|R2XYDBMHUVJCSX|          2|            0|          1|   N|                Y|
|R1L94ESVC657A9|          5|            0|          0|   N|                Y|
|R3DI4B8LDWFQ3K|          1|            3|          4|   N|                Y|
|R1G7VV4HCXUQQL|          5|            0|          0|   N|                Y|
|R3TTNYN1TAMUSU|          5|            0|          0|   N|     

In [46]:
from pyspark.sql.functions import count
from pyspark.sql.functions import col

In [47]:
# 1. Filter and create new df to retrieve all rows where total_votes => 20
total_votes_df = df.filter("total_votes >= 20").select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|
|R1M8EEGH2TVS0Q|          1|            6|         29|   N|                Y|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|
|R3JF2SBRHZITRZ|          5|           24|         26|   N|                N|
| RLJN73T8WUYIR|          5|          124|        128|   N|     

In [48]:
# Create new df to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
new_df = total_votes_df.withColumn("result", total_votes_df.helpful_votes/total_votes_df.total_votes)
new_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|             result|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y| 0.8043478260869565|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y| 0.9646464646464646|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|               0.98|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y| 0.9642857142857143|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|                1.0|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N| 0.9523809523809523|
|R1M8EEGH2TVS0Q|          1|            6|         29|   N|                Y|0.20689655172413793|
| RK8MUEWEUGV6V|    

In [49]:
# Create new df to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
new50_df = new_df.filter("result >= .5")
new50_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|            result|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|0.8043478260869565|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|0.9646464646464646|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|              0.98|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|0.9642857142857143|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|               1.0|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|0.9523809523809523|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|0.9292035398230089|
|R3JF2SBRHZITRZ|          5|  

In [50]:
# Create a new DataFrame that retrieves all the rows where vine == 'Y'
vine_yes_df = new50_df.filter(new50_df.vine == "Y")
vine_yes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|            result|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RG70K1HMY4LXX|          4|           37|         44|   Y|                N|0.8409090909090909|
|R1DHGTNXDXJ0GB|          5|           78|         93|   Y|                N|0.8387096774193549|
|R3B5S3OSA8N6AO|          5|           38|         43|   Y|                N|0.8837209302325582|
|R29X0YBS0SBIUP|          5|           56|         74|   Y|                N|0.7567567567567568|
|R1ZEDMCU6NYC8Q|          3|           26|         27|   Y|                N|0.9629629629629629|
| RVBJM1LUI9X9K|          5|           27|         28|   Y|                N|0.9642857142857143|
| R1KASHD9QSDMV|          4|           18|         21|   Y|                N|0.8571428571428571|
|R1M3LATUI38MK0|          5|  

In [51]:
# # Create a new DataFrame that retrieves all the rows where vine == 'N'
vine_no_df = new50_df.filter(new50_df.vine == "N")
vine_no_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|            result|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|0.8043478260869565|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|0.9646464646464646|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|              0.98|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|0.9642857142857143|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|               1.0|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|0.9523809523809523|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|0.9292035398230089|
|R3JF2SBRHZITRZ|          5|  

In [58]:
# PAID: Determine the total number of paid reviews
paid_total_reviews = vine_yes_df.count()
paid_total_reviews

497

In [64]:
# PAID: Total 5 star paid reviews
filter_paid_5star_reviews = vine_yes_df.filter("star_rating == 5")
paid_5star_reviews = filter_paid_5star_reviews.count()
paid_5star_reviews

220

In [65]:
# PAID: Percentage of 5-star paid reviews
percentage_5star_reviews = (paid_5star_reviews/paid_total_reviews) * 100
percentage_5star_reviews

44.26559356136821

In [66]:
# UNPAID: Determine the total number of unpaid reviews
unpaid_total_reviews = vine_no_df.count()
unpaid_total_reviews

120863

In [67]:
# UNPAID: Total 5 star unpaid reviews
filter_unpaid_5star_reviews = vine_no_df.filter("star_rating == 5")
unpaid_5star_reviews = filter_unpaid_5star_reviews.count()
unpaid_5star_reviews

74470

In [68]:
# UNPAID: Percentage of 5-star unpaid reviews
percentage_5star_unpaid_reviews = (unpaid_5star_reviews/unpaid_total_reviews) * 100
percentage_5star_unpaid_reviews

61.615217229425056